$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [2]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [2]:
#!pip install --force-reinstall numpy
#!pip install --force-reinstall scikit-surprise


  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.m

In [3]:
#!pip uninstall -y numpy scikit-surprise
#!pip install numpy==1.24.4
#!pip install scikit-surprise --no-binary :all:


Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
Found existing installation: scikit-surprise 1.1.4
Uninstalling scikit-surprise-1.1.4:
  Successfully uninstalled scikit-surprise-1.1.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which 

  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl


In [3]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.accuracy import rmse,mse,mae

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [4]:
data = pd.read_csv("/content/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [5]:
utility_matrix = data.pivot(index='userID', columns='songID',values='rating')
utility_matrix = utility_matrix.fillna(0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [6]:
k=3
no_of_users = utility_matrix.shape[0]
no_of_items = utility_matrix.shape[1]
U = np.random.random((no_of_users, k))
M = np.random.random((k, no_of_items))

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [7]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """
    U = U.copy()
    M = M.copy()

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = M.shape[0]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                    eij =  R[i,j] - (U[i] @ M.T[j])  # Calculate the error in prediction
                    for d in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, d] = U[i, d] + lr*(2*eij*M[d, j] - beta * U[i, d])
                        M[d, j] = M[d, j] + lr*(2*eij*U[i, d] - beta * M[d, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value

                    first_part =(U[i] @ M.T[j] - R[i,j]) ** 2  # calculate the first part of the error
                    second_part = 0 # Initialize a variable to accumelate the second part of the error

                    for d in range(K): # Loop over each latent features dimension
                        second_part += ((beta / 2) * (U[i,d] ** 2 + M[d,j] ** 2))

                    e += first_part + second_part # accumelate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [8]:
def matrix_factorization_vectorized(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    U = U.copy()
    M = M.copy()

    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    nonzero_indices = np.nonzero(R)

    for epoch in range(epochs):
        # Calculate error matrix E
        E = R - np.dot(U, M)
        E[R == 0] = 0

        # Update U and M
        U_grad = 2 * np.dot(E, M.T) - beta * U
        M_grad = 2 * np.dot(U.T, E) - beta * M

        np.clip(U_grad, -1e10, 1e10, out=U_grad)
        np.clip(M_grad, -1e10, 1e10, out=M_grad)

        U += lr * U_grad
        M += lr * M_grad

        # Calculate total error
        first_part = np.sum(E ** 2)
        second_part = beta * (np.sum(U ** 2) + np.sum(M ** 2))
        error = first_part + second_part

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {error}')

        # Check for convergence
        if error < 0.001:
            break

    return U, M

## `iii` Use the Function to to Optimize the $U$ and $V$

In [10]:
U_1, M_1 = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=50, lr=0.01, beta=0.001)


Epoch 1/50: Total Error = 117522.37237902026
Epoch 2/50: Total Error = 65543.23272046151
Epoch 3/50: Total Error = 44512.314021462706
Epoch 4/50: Total Error = 35176.686860625894
Epoch 5/50: Total Error = 30398.621191775335
Epoch 6/50: Total Error = 27352.71041826281
Epoch 7/50: Total Error = 24920.15814460477
Epoch 8/50: Total Error = 22709.582649156535
Epoch 9/50: Total Error = 20633.9854063459
Epoch 10/50: Total Error = 18710.561865516625
Epoch 11/50: Total Error = 16971.350029249017
Epoch 12/50: Total Error = 15432.543092773287
Epoch 13/50: Total Error = 14091.73855109855
Epoch 14/50: Total Error = 12934.503028318893
Epoch 15/50: Total Error = 11940.850424419907
Epoch 16/50: Total Error = 11089.207380625467
Epoch 17/50: Total Error = 10358.595601019497
Epoch 18/50: Total Error = 9729.97264869523
Epoch 19/50: Total Error = 9187.03612976369
Epoch 20/50: Total Error = 8716.462904917844
Epoch 21/50: Total Error = 8307.615282149896
Epoch 22/50: Total Error = 7951.9106367777
Epoch 23/50:

In [12]:
U_2, M_2 = matrix_factorization_vectorized(R=utility_matrix.values, U=U, M=M, epochs=50, lr=0.01, beta=0.001)


Epoch 1/50: Total Error = 703246.9411297546
Epoch 2/50: Total Error = 241464185.7096196
Epoch 3/50: Total Error = 2.512077884127e+16
Epoch 4/50: Total Error = 1.6932403854801602e+36
Epoch 5/50: Total Error = 6.338200229126178e+34
Epoch 6/50: Total Error = 1.6932403854801602e+36
Epoch 7/50: Total Error = 6.338200229126178e+34
Epoch 8/50: Total Error = 1.6932403854801602e+36
Epoch 9/50: Total Error = 6.338200229126178e+34
Epoch 10/50: Total Error = 1.6932403854801602e+36
Epoch 11/50: Total Error = 6.338200229126178e+34
Epoch 12/50: Total Error = 1.6932403854801602e+36
Epoch 13/50: Total Error = 6.338200229126178e+34
Epoch 14/50: Total Error = 1.6932403854801602e+36
Epoch 15/50: Total Error = 6.338200229126178e+34
Epoch 16/50: Total Error = 1.6932403854801602e+36
Epoch 17/50: Total Error = 6.338200229126178e+34
Epoch 18/50: Total Error = 1.6932403854801602e+36
Epoch 19/50: Total Error = 6.338200229126178e+34
Epoch 20/50: Total Error = 1.6932403854801602e+36
Epoch 21/50: Total Error = 6.33

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [13]:
# Find the index of user 199988 in the U matrix
user_id = 199988
user_ids = utility_matrix.index
user_idx = np.where(user_ids == user_id)[0][0]

# Compute predicted ratings for user 199988
predicted_ratings = np.dot(U_1[user_idx], M_1)

# Find the indices of already rated items
rated_items = np.nonzero(utility_matrix.loc[user_id].values)[0]

# Set predicted ratings of already rated items to -1 (to exclude them from top-K recommendations)
predicted_ratings[rated_items] = -1

# Get the indices and values of top-K items
top_K_item_ids = np.argsort(predicted_ratings)[-5:][::-1]
top_K_item_ratings = np.sort(predicted_ratings)[-5:][::-1]

print(f"Top-5 recommendations for User {user_id}:")
for i,(id,rating) in enumerate(zip(utility_matrix.columns[top_K_item_ids],top_K_item_ratings)):
    print(f"\t -Top {i+1} Song: {id} (Predicted Rating: {rating})")

Top-5 recommendations for User 199988:
	 -Top 1 Song: 55240 (Predicted Rating: 6.25611151456272)
	 -Top 2 Song: 68572 (Predicted Rating: 6.032352155450793)
	 -Top 3 Song: 60888 (Predicted Rating: 5.9862230404145285)
	 -Top 4 Song: 94535 (Predicted Rating: 5.839644194892192)
	 -Top 5 Song: 86341 (Predicted Rating: 5.7077523334050895)


Top 5 Recommended Items for User 199988:
	- Top 1 Song: 122065 (Predicted Rating: 6.006507760135539)
	- Top 2 Song: 125557 (Predicted Rating: 5.819802641310125)
	- Top 3 Song: 52611 (Predicted Rating: 5.740148229067794)
	- Top 4 Song: 79622 (Predicted Rating: 5.701478248255747)
	- Top 5 Song: 71582 (Predicted Rating: 5.691077461653294)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [14]:
reader = Reader()
dataset = Dataset.load_from_df(data, reader)

In [15]:
train_data, test_data = train_test_split(dataset, test_size=0.2)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [16]:
# Biased Model
model_1= SVD()

In [17]:
# Non-Biased Model
model_2 = SVD(biased=False)


## `iii` Fit each Model on Training Data

In [18]:
# Biased Model
model_1.fit(train_data)


In [19]:
# Non-Biased Model
model_2.fit(train_data)


## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [20]:
# Biased Model
predictions_1 = model_1.test(test_data)
print(rmse(predictions_1))
print(mse(predictions_1))
print(mae(predictions_1))

RMSE: 1.4997
1.499716625745729
MSE: 2.2491
2.249149957538155
MAE:  1.3000
1.300027129334421


In [21]:
# Non-Biased Model
predictions_2 = model_2.test(test_data)
print(rmse(predictions_2))
print(mse(predictions_2))
print(mae(predictions_2))

RMSE: 2.0910
2.090953196104404
MSE: 4.3721
4.372085268299223
MAE:  1.7410
1.7410354942097093


## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [22]:
# Biased Model
users_1 = np.array([item.uid for item in predictions_1]).reshape(-1,1)
items_1 = np.array([item.iid for item in predictions_1]).reshape(-1,1)
real_1 = np.array([item.r_ui for item in predictions_1]).reshape(-1,1)
predicted_1 = np.array([item.est for item in predictions_1]).reshape(-1,1)

modified_predicted_1 = np.concatenate([users_1,items_1,real_1,predicted_1],axis=1)

In [23]:
# Non-Biased Model
User_ID = 199988
df_biased = pd.DataFrame(modified_predicted_1,columns=['UserID','SongID','real_rating','Predicted_rating'])
df_biased.drop(columns=['UserID'],inplace=True)

#df_filtered = df_biased.loc[(df_biased['UserID'] == User_ID)]
#df_filtered_sorted = df_filtered.sort_values('Predicted_rating', ascending=False)
#df_filtered_sorted

df_biased_sorted = df_biased.sort_values('Predicted_rating', ascending=False)
df_biased_sorted.head(10)

,SongID,real_rating,Predicted_rating
1637,60465.0,3.0,5.000000
8204,2263.0,5.0,5.000000
3471,55240.0,5.0,5.000000
13173,126757.0,5.0,5.000000
6951,13859.0,5.0,5.000000
83,52611.0,4.0,5.000000
5880,71582.0,5.0,4.956068
10375,71582.0,5.0,4.927268
5211,22763.0,1.0,4.905051
3124,71582.0,5.0,4.903304


In [24]:
users_2 = np.array([item.uid for item in predictions_2]).reshape(-1,1)
items_2 = np.array([item.iid for item in predictions_2]).reshape(-1,1)
real_2 = np.array([item.r_ui for item in predictions_2]).reshape(-1,1)
predicted_2 = np.array([item.est for item in predictions_2]).reshape(-1,1)

modified_predicted_2 = np.concatenate([users_2,items_2,real_2,predicted_2],axis=1)

In [25]:
# Non-Biased Model
User_ID = 199988
df_non_biased = pd.DataFrame(modified_predicted_2,columns=['UserID','SongID','real_rating','Predicted_rating'])
df_non_biased.drop(columns=['UserID'],inplace=True)
df_non_biased_sorted = df_non_biased.sort_values('Predicted_rating', ascending=False)
df_non_biased_sorted.head(10)

,SongID,real_rating,Predicted_rating
10384,42906.0,5.0,3.464766
0,12709.0,5.0,3.452460
8489,19299.0,4.0,3.452460
8477,94535.0,5.0,3.452460
8478,40712.0,2.0,3.452460
8479,79622.0,4.0,3.452460
8480,12709.0,5.0,3.452460
8483,48731.0,5.0,3.452460
8484,125557.0,2.0,3.452460
8487,105421.0,3.0,3.452460


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$